# Run the forward model

emissions through to temperature

A TODO here is to take in an emissions file e.g. from RCMIP, and perform the unit conversion automatically into the FaIR desired units.

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import scipy.linalg
import scipy.stats
from scipy.interpolate import interp1d
import json
from tqdm import tqdm

from fair21.constants.gases import molwt, burden_per_emission, lifetime, gas_list
from fair21.constants.general import NBOX, EARTH_RADIUS, SECONDS_PER_YEAR, DOUBLING_TIME_1PCT
from fair21.defaults import slcf_list, minor_gas_list, gas_list_excl_co2_ch4, montreal_gas_list, run_mode, valid_run_modes
from fair21.defaults.aerosol.radiation import radiative_efficiency as radiative_efficiency_erfari
from fair21.defaults.aerosol.cloud import beta, shape
from fair21.defaults.forcing import tropospheric_adjustment
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon,
    iirf_max,
    radiative_efficiency as radiative_efficiency_ghg
)
from fair21.defaults.landuse import forcing_from_cumulative_co2_afolu
from fair21.defaults.short_lived_forcers import pre_industrial_emissions
from fair21.exceptions import (
    IncompatibleConfigError,
    InvalidRunModeError,
    MissingInputError,
    NonNumericInputError,
    ScenarioLengthMismatchError,
    TimeNotDefinedError,
    UndefinedSpeciesError,
    UnknownRunModeError,
)
from fair21.forcing import forcing_from_input
from fair21.forcing.aerosol import radiation, cloud
from fair21.forcing.black_carbon_on_snow import linear as bcsnow_linear
from fair21.forcing.contrail import linear_from_aviation_nox
from fair21.forcing.ghg import meinshausen, linear as ghg_linear
from fair21.forcing.ozone import thornhill_skeie
from fair21.forcing.landuse import from_cumulative_co2
from fair21.forcing.stratospheric_water_vapour import linear as linear_stwv

from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha,
)
from fair21.gas_cycle.forward import step_concentration

In [ ]:
# grab some emissions
emissions = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for specie in gas_list + slcf_list + ["CO2|MAGICC AFOLU", "NOx|MAGICC Fossil and Industrial|Aircraft"]:
    gas_rcmip_name = specie.replace("-", "")
    emissions[specie] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()
    
# CO2 and N2O units need to behave
emissions["CO2|MAGICC AFOLU"] = emissions["CO2|MAGICC AFOLU"] / 1000
emissions["CO2"] = emissions["CO2"] / 1000
emissions["N2O"] = emissions["N2O"] / 1000

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp245-mean-temperature.csv')
ssp245_temperature_rfmip = df['temperature'].values

In [ ]:
def _check_inputs(inputs, time):
    
    if len(inputs)==0:
        raise MissingInputError("At least one species is required")

    # check to see if species are same length
    n_timesteps = len(time)
    for ispec, specie in enumerate(inputs.keys()):
        len_input = len(inputs[specie])
        if len_input != n_timesteps:
            raise ScenarioLengthMismatchError("Species {} has {} timesteps, which differs from the time vector that has {}".format(specie, len_input, n_timesteps))

In [ ]:
def _check_config(run_mode):
    for treatment in run_mode:
        if treatment not in valid_run_modes:
            raise UnknownRunModeError("{} is not recognised as a run_mode".format(treatment))
        if run_mode[treatment] not in valid_run_modes[treatment]:
            raise InvalidRunModeError("{} is not a valid run mode for {}. Valid options are {}".format(run_mode[treatment], treatment, valid_run_modes[treatment]))

In [ ]:
## this has been tested so we'll comment out to allow notebook to run without errors

# import copy
# bad_run_mode = copy.copy(run_mode)\
# bad_run_mode['Volcanic'] = 'Exploding!'
# bad_run_mode['Cosmic Rays'] = 'forcing'
# _check_config(bad_run_mode)

In [ ]:
def _make_time_deltas(time):
    time_inner_bounds = 0.5*(time[1:] + time[:-1])
    time_lower_bound = time[0] - (time_inner_bounds[0] - time[0])
    time_upper_bound = time[-1] + (time[-1] - time_inner_bounds[-1])
    time_bounds = np.concatenate(([time_lower_bound], time_inner_bounds, [time_upper_bound]))
    time_deltas = np.diff(time_bounds)
    return time_deltas

In [ ]:
def ensure_numeric_array(obj):
    obj = np.atleast_1d(obj)
    if not np.issubdtype(obj.dtype, np.number):
        raise NonNumericInputError("{} is not a numeric array".format(obj))
    
    return obj

In [ ]:
def calc_concentration_no_feedbacks(
    emissions, 
    burden_per_emission,
    lifetime,
    pre_industrial_concentration,
    timestep,
    natural_emissions_adjustment
):
    n_timesteps = len(emissions)
    concentration = np.ones(n_timesteps) * np.nan
    airborne_emissions = np.ones(n_timesteps) * np.nan
    ae_timestep = 0
    gas_boxes = 0
    for i in range(n_timesteps):
        concentration[i], gas_boxes, ae_timestep = step_concentration(
            emissions[i], 
            gas_boxes,
            ae_timestep,
            burden_per_emission,
            lifetime,
            alpha_lifetime=1,
            pre_industrial_concentration=pre_industrial_concentration,
            timestep=timestep[i],
            partition_fraction=1,
            natural_emissions_adjustment=natural_emissions_adjustment
        )
        airborne_emissions[i] = ae_timestep
        #print(concentration[i], ae_timestep)
    return concentration, airborne_emissions

In [ ]:
run_mode

In [ ]:
class EnergyBalanceModel:
    """Energy balance model that converts forcing to temperature.

    The energy balance model is converted to an impulse-response formulation
    (hence the IR part of FaIR) to allow efficient evaluation. The benefits of
    this are increased as once derived, the "layers" of the energy balance
    model do not communicate with each other.

    Attributes
    ----------

    References
    ----------

    .. [1] Leach et al. (2021): https://doi.org/10.5194/gmd-14-3007-2021

    .. [2] Cummins et al. (2020): https://doi.org/10.1175/JCLI-D-19-0589.1

    .. [3] Tsutsui (2017): https://doi.org/10.1007/s10584-016-1832-9

    .. [4] Geoffroy et al. (2013): https://doi.org/10.1175/JCLI-D-12-00196.1
    """
    
    def __init__(self, **kwargs):
        """Initialise the EnergyBalanceModel.

        Parameters
        ----------
        **kwargs : dict, optional
            Parameters to run the energy balance model with.

            ocean_heat_capacity : `np.ndarray`
                Ocean heat capacity of each layer (top first), W m-2 yr K-1
            ocean_heat_transfer : `np.ndarray`
                Heat exchange coefficient between ocean layers (top first). The
                first element of this array is akin to the climate feedback
                parameter, with the convention that stabilising feedbacks are
                positive (opposite to most climate sensitivity literature).
                W m-2 K-1
            deep_ocean_efficacy : float
                efficacy of deepest ocean layer. See e.g. [1]_.
            forcing_4co2 : float
                effective radiative forcing from a quadrupling of atmospheric
                CO2 concentrations above pre-industrial.
            stochastic_run : bool
                Activate the stochastic variability component from [2]_.
            sigma_eta : float
                Standard deviation of stochastic forcing component from [2]_.
            sigma_xi : float
                Standard deviation of stochastic disturbance applied to surface
                layer. See [2]_.
            gamma_autocorrelation : float
                Stochastic forcing continuous-time autocorrelation parameter. 
                See [2]_.
            seed : int or None
                Random seed to use for stochastic variability.
                
        References
        ----------
        .. [1] Geoffroy, O., Saint-Martin, D., Bellon, G., Voldoire, A., Olivié,
            D. J. L., & Tytéca, S. (2013). Transient Climate Response in a Two-
            Layer Energy-Balance Model. Part II: Representation of the Efficacy
            of Deep-Ocean Heat Uptake and Validation for CMIP5 AOGCMs, Journal 
            of Climate, 26(6), 1859-1876

        .. [2] Cummins, D. P., Stephenson, D. B., & Stott, P. A. (2020). Optimal
            Estimation of Stochastic Energy Balance Model Parameters, Journal of
            Climate, 33(18), 7909-7926.
        """
        self.ocean_heat_capacity = kwargs.get('ocean_heat_capacity', np.array([5, 20, 100]))
        self.ocean_heat_transfer = kwargs.get('ocean_heat_transfer', np.array([1, 2, 1]))
        self.deep_ocean_efficacy = kwargs.get('deep_ocean_efficacy', 1)
        self.forcing_4co2 = kwargs.get('forcing_4co2', 8)
        self.stochastic_run = kwargs.get('stochastic_run', False)
        self.sigma_eta = kwargs.get('sigma_eta', 0.5)
        self.sigma_xi = kwargs.get('sigma_xi', 0.5)
        self.gamma_autocorrelation = kwargs.get('gamma_autocorrelation', 2)
        self.seed = kwargs.get('seed', None)
        self.temperature = kwargs.get('temperature', np.zeros((1, NBOX)))
        
    def _eb_matrix(self):
        """Define the matrix of differential equations.

        Returns
        -------
        eb_matrix_eigenvalues : `np.ndarray`
            1D array of eigenvalues of the energy balance matrix.
        eb_matrix_eigenvectors : `np.ndarray`
            2D array of eigenvectors (an array of 1D eigenvectors) of the
            energy balance matrix.
        """
        eb_matrix = np.array(
            [
                [
                    -(self.ocean_heat_transfer[0]+self.ocean_heat_transfer[1])/self.ocean_heat_capacity[0],
                    self.ocean_heat_transfer[1]/self.ocean_heat_capacity[0], 
                    0
                ],
                [
                    self.ocean_heat_transfer[1]/self.ocean_heat_capacity[1],
                    -(self.ocean_heat_transfer[1]+self.deep_ocean_efficacy*self.ocean_heat_transfer[2])/self.ocean_heat_capacity[1],
                    self.deep_ocean_efficacy*self.ocean_heat_transfer[2]/self.ocean_heat_capacity[1]
                ],
                [
                    0, 
                    self.ocean_heat_transfer[2]/self.ocean_heat_capacity[2],
                    -self.ocean_heat_transfer[2]/self.ocean_heat_capacity[2]
                ]
            ]
        )
        return(eb_matrix)
    
        
    def impulse_response(self):
        """Converts the energy balance to impulse response."""
        eb_matrix = self._eb_matrix()

        # calculate the eigenvectors and eigenvalues, these are the timescales of responses
        eb_matrix_eigenvalues, eb_matrix_eigenvectors = scipy.linalg.eig(eb_matrix)
        self.timescales = -1/(np.real(eb_matrix_eigenvalues))
        self.response_coefficients = self.timescales * (eb_matrix_eigenvectors[0,:] * scipy.linalg.inv(eb_matrix_eigenvectors)[:,0]) / self.ocean_heat_capacity[0]

        
    def emergent_parameters(self):
        """Calculates emergent parameters from the energy balance parameters."""
        # requires impulse response step
        if not hasattr(self, 'timescales'):
            self.impulse_response()
        self.ecs = self.forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(self.response_coefficients)
        self.tcr = self.forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(
            self.response_coefficients*(
                1 - self.timescales/DOUBLING_TIME_1PCT * (
                    1 - np.exp(-DOUBLING_TIME_1PCT/self.timescales)
                )
            )
        )
    
    def add_forcing(self, forcing, time):
        self.forcing = forcing
        self.time = time
    
    
    def run(self):
        # internal variables
        nmatrix = NBOX + self.stochastic_run
        n_timesteps = len(self.time)
        
        # Define the forcing vector
        forcing_vector = np.array([1/self.ocean_heat_capacity[0], 0, 0])
    
        # Prepend eb_matrix with stochastic terms if this is a stochastic run: Cummins et al. (2020) eqs. 13 and 14
        eb_matrix = self._eb_matrix()
        if self.stochastic_run:
            eb_matrix = np.insert(eb_matrix, 0, np.zeros(NBOX), axis=0)
            prepend_col = np.zeros(nmatrix)
            prepend_col[0] = -self.gamma_autocorrelation
            prepend_col[1] = 1/self.ocean_heat_capacity[0]
            eb_matrix = np.insert(eb_matrix, 0, prepend_col, axis=1)
            forcing_vector = np.zeros(nmatrix)
            forcing_vector[0] = self.gamma_autocorrelation
    
        # Calculate the matrix exponential
        eb_matrix_d = scipy.linalg.expm(eb_matrix)
    
        # Solve for temperature
        forcing_vector_d = scipy.linalg.solve(eb_matrix, (eb_matrix_d - np.identity(nmatrix)) @ forcing_vector)
    
        # define stochastic matrix
        stochastic_d = np.zeros((n_timesteps, nmatrix))
    
        # stochastic stuff
        if self.stochastic_run:
            q_mat = np.zeros((nmatrix, nmatrix))
            q_mat[0,0] = self.sigma_eta**2
            q_mat[1,1] = (self.sigma_xi/self.ocean_heat_capacity[0])**2
            ## use Van Loan (1978) to compute the matrix exponential
            h_mat = np.zeros((nmatrix*2, nmatrix*2))
            h_mat[:nmatrix,:nmatrix] = -eb_matrix
            h_mat[:nmatrix,nmatrix:] = q_mat
            h_mat[nmatrix:,nmatrix:] = eb_matrix.T
            g_mat = scipy.linalg.expm(h_mat)
            q_mat_d = g_mat[nmatrix:,nmatrix:].T @ g_mat[:nmatrix,nmatrix:]
            q_mat_d = q_mat_d.astype(np.float64)
            stochastic_d = scipy.stats.multivariate_normal.rvs(
                size=n_timesteps, mean=np.zeros(nmatrix), cov=q_mat_d, random_state=self.seed
            )

        solution = np.zeros((n_timesteps, nmatrix))
        solution[0, :] = self.temperature[0, :]
        for i in range(1, n_timesteps):
            solution[i, :] = eb_matrix_d @ solution[i-1, :] + forcing_vector_d * self.forcing[i-1] + stochastic_d[i-1, :]

        if self.stochastic_run:
            self.temperature = solution[:, 1:]
            self.stochastic_forcing = solution[:, 0]
        else:
            self.temperature = solution
        self.toa_imbalance = self.forcing - self.ocean_heat_transfer[0]*self.temperature[:,0] + (1 - self.deep_ocean_efficacy) * self.ocean_heat_transfer[2] * (self.temperature[:,1] - self.temperature[:,2])
        

    def step_temperature(self, temperature_boxes_old, forcing):
        """Timestep the temperature forward.
        
        Unlike the `run` instance, this increments a single timestep, and should
        prevent inverting a matrix each time.
        """
        
        # forcing_vector and eb_matrix should both be determininstic if not running stochastically
        # and we probably can't make this quick if we were
        # actually we probably can...
        temperature_new = eb_matrix_d @ temperature_old + forcing_vector_d * forcing
        
        
        # think we'd be better with step_temperature here
        # OHC now needs to be after the fact
        #self.ocean_heat_content_change = np.cumsum(self.toa_imbalance * np.concatenate(([0], np.diff(self.time)))) * EARTH_RADIUS**2 * 4 * np.pi * SECONDS_PER_YEAR

In [ ]:
class FAIR:
    
    def __init__(self, **kwargs):
        # initialise the model
        self.emissions = {}
        self.concentration = {}
        self.airborne_emissions = {}
        self.cumulative_emissions = {}
        self.effective_radiative_forcing = {}
        self.iirf_0 = kwargs.get('iirf_0', iirf_0)
        self.iirf_cumulative = kwargs.get('iirf_cumulative', iirf_cumulative)
        self.iirf_temperature = kwargs.get('iirf_temperature', iirf_temperature)
        self.iirf_airborne = kwargs.get('iirf_airborne', iirf_airborne)
        self.iirf_horizon = kwargs.get('iirf_horizon', iirf_horizon)
        self.iirf_max = kwargs.get('iirf_max', iirf_max)
        self.lifetime = kwargs.get('lifetime', lifetime)
        self.burden_per_emission = kwargs.get('burden_per_emission', burden_per_emission)
        self.pre_industrial_concentration = kwargs.get('pre_industrial_concentration', pre_industrial_concentration)
        self.pre_industrial_emissions = kwargs.get('pre_industrial_emissions', pre_industrial_emissions)
        self.natural_emissions_adjustment = kwargs.get('natural_emissions_adjustment', natural_emissions_adjustment)
        self.radiative_efficiency_ghg = kwargs.get('radiative_efficiency_ghg', radiative_efficiency_ghg)
        self.partition_fraction = kwargs.get('partition_fraction', partition_fraction)
        self.radiative_efficiency_erfari = kwargs.get('radiative_efficiency_erfari', radiative_efficiency_erfari['AR6'])
        self.beta = kwargs.get('beta', beta['AR6'])
        self.shape_sulfur = kwargs.get('shape_sulfur', shape['AR6']['Sulfur'])
        self.shape_bcoc = kwargs.get('shape_bcoc', shape['AR6']['BC+OC'])
        self.forcing_from_cumulative_co2_afolu = kwargs.get('forcing_from_cumulative_co2_afolu', forcing_from_cumulative_co2_afolu['AR6'])
        self.tropospheric_adjustment = kwargs.get('tropospheric_adjustment', tropospheric_adjustment)
        self.run_mode = kwargs.get('run_mode', run_mode)
        self.calculate_aerosol_forcing = kwargs.get('calculate_aerosol_forcing', True)
        self.time = kwargs.get('time')
        self.volcanic_forcing = kwargs.get('volcanic_forcing', 0)
        self.solar_forcing = kwargs.get('solar_forcing', 0)
        self.time_minus1 = kwargs.get('time_minus1', None)
        self.ocean_heat_capacity = kwargs.get('ocean_heat_capacity', np.array([5, 20, 100]))
        self.ocean_heat_transfer = kwargs.get('ocean_heat_transfer', np.array([1, 2, 1]))
        self.deep_ocean_efficacy = kwargs.get('deep_ocean_efficacy', 1)
        self.stochastic_run = kwargs.get('stochastic_run', False)
        self.sigma_eta = kwargs.get('sigma_eta', 0.5)
        self.sigma_xi = kwargs.get('sigma_xi', 0.5)
        self.gamma_autocorrelation = kwargs.get('gamma_autocorrelation', 2)
        self.seed = kwargs.get('seed', None)
        
    def add_emissions(self, emissions):
        for specie in emissions:
            emissions[specie] = ensure_numeric_array(emissions[specie])
        self.emissions.update(emissions)
        
    def add_concentration(self, concentration):
        for specie in concentration:
            concentration[specie] = ensure_numeric_array(concentration[specie])
        self.concentration.update(concentration)
        
    def add_forcing(self, effective_radiative_forcing):
        for forcer in effective_radiative_forcing:
            effective_radiative_forcing[forcer] = ensure_numeric_array(effective_radiative_forcing[forcer])
        self.effective_radiative_forcing.update(effective_radiative_forcing)
        
    def define_time(self, time):
        self.time = ensure_numeric_array(time)
        
    def prescribe_temperature(self, temperature):
        self.temperature = ensure_numeric_array(temperature)
    
    def run(self):
        """Run FaIR."""
        
        # run initial sense checks on inputs.
        if not hasattr(self, 'time'):
            raise TimeNotDefinedError("Time vector is not defined")
        n_timesteps = len(self.time)
        time_deltas = _make_time_deltas(self.time)
        if not hasattr(self, 'time_minus1'):  # guess what initialisation time is; should log a warning here
            self.time_minus1 = self.time[0] - time_deltas[0]
        _check_inputs(self.emissions, self.time)
        _check_config(self.run_mode)
        
        # create temperature array of zeros if temperature not prescribed
        temperature_prescribed = True
        if not hasattr(self, 'temperature'):
            self.temperature = np.zeros_like(self.time)    
            temperature_prescribed = False
        
        # Initialise required forcing outputs
        if self.run_mode['Stratospheric water vapour'] == 'emissions':
            self.effective_radiative_forcing['Stratospheric water vapour'] = np.ones(n_timesteps) * np.nan
        if self.run_mode['Black carbon on snow'] == 'emissions':
            self.effective_radiative_forcing['Black carbon on snow'] = np.ones(n_timesteps) * np.nan
        if self.run_mode['Ozone'] == 'emissions':
            ozone_components = [
                'Ozone|Emitted Gases|CH4',
                'Ozone|Emitted Gases|N2O',
                'Ozone|Emitted Gases|Montreal Gases',
                'Ozone|Emitted Gases|VOC',
                'Ozone|Emitted Gases|NOx',
                'Ozone|Emitted Gases|CO',
                'Ozone|Emitted Gases',
                'Ozone|Temperature Feedback',
                'Ozone'
            ]
            for component in ozone_components:
                self.effective_radiative_forcing[component] = np.ones(n_timesteps) * np.nan
        # TODO: option to just run forcing driven
        # I don't like initialising with zeros, but will avoid an if-statement
        self.effective_radiative_forcing['Total'] = np.zeros(n_timesteps)
        
        # Initialise energy balance model: and grab matrix
        ebm = EnergyBalanceModel(
            ocean_heat_capacity=self.ocean_heat_capacity,
            ocean_heat_transfer=self.ocean_heat_transfer,
            deep_ocean_efficacy=self.deep_ocean_efficacy,
            stochastic_run=self.stochastic_run,
            sigma_eta=self.sigma_eta,
            sigma_xi=self.sigma_xi,
            gamma_autocorrelation=self.gamma_autocorrelation,
            seed=self.seed,
        )
        
        ebm._eb_matrix
        
        # quickest to calculate every forcer that is not temperature-dependent upfront:
        # 1. non-CO2, non-CH4 GHGs
        if self.run_mode['Other greenhouse gases'] == 'emissions':
            
            for specie in list(set(self.emissions) & set(minor_gas_list)):
                self.cumulative_emissions[specie] = np.cumsum(self.emissions[specie])
                self.concentration[specie], self.airborne_emissions[specie] = calc_concentration_no_feedbacks(
                    self.emissions[specie],
                    self.burden_per_emission[specie],
                    self.lifetime[specie],
                    self.pre_industrial_concentration[specie],
                    time_deltas,
                    self.natural_emissions_adjustment[specie]
                )
            
#        if self.run_mode['other_greenhouse_gases'] not in ('forcing', None):
                # can't do N2O in this step
                self.effective_radiative_forcing.update(
                    ghg_linear(
                        self.concentration,
                        self.pre_industrial_concentration,
                        self.radiative_efficiency_ghg,
                        self.tropospheric_adjustment,
                    )
                )

        if 'N2O' in self.emissions and self.run_mode['N2O'] == 'emissions':
            specie='N2O'
            self.effective_radiative_forcing[specie] = np.ones(n_timesteps) * np.nan
            self.cumulative_emissions[specie] = np.cumsum(self.emissions[specie])
            self.concentration[specie], self.airborne_emissions[specie] = calc_concentration_no_feedbacks(
                self.emissions[specie],
                self.burden_per_emission[specie],
                self.lifetime[specie],
                self.pre_industrial_concentration[specie],
                time_deltas,
                self.natural_emissions_adjustment[specie]
            )
        
        # 2. aerosols : if we ever add a temperature feedback this will need to be moved.
        if self.run_mode['Aerosol'] == 'emissions':
            self.effective_radiative_forcing.update(
                radiation.linear(
                    self.emissions,
                    self.pre_industrial_emissions,
                    self.radiative_efficiency_erfari,
                )
            )
            
            self.effective_radiative_forcing['Aerosol|Aerosol-cloud interactions'] = (
                cloud.smith2021(
                    self.emissions,
                    self.pre_industrial_emissions,
                    self.beta,
                    self.shape_sulfur,
                    self.shape_bcoc,
                )
            )
            self.effective_radiative_forcing['Aerosol'] = (
                self.effective_radiative_forcing['Aerosol|Aerosol-cloud interactions'] +
                self.effective_radiative_forcing['Aerosol|Aerosol-radiation interactions']
            )
        
        # 3. contrails
        if self.run_mode['Contrails'] == 'emissions':
            self.effective_radiative_forcing['Contrails'] = (
                linear_from_aviation_nox(
                    emissions,
                )
            )
        
        # 4. BC on snow
        if self.run_mode['Black carbon on snow'] == 'emissions':
            self.effective_radiative_forcing['Black carbon on snow'] = (
                bcsnow_linear(
                    emissions,
                )
            )
        
        # 5. land use change (in the forward model)
        # this treatment at present can only be run in CO2 forward emissions mode, 
        # because we need to calculate a fossil/AFOLU split if not.
        if self.run_mode['Land use'] == 'emissions':
            if self.run_mode['CO2'] != 'emissions':
                raise IncompatibleConfigError("To run landuse forcing in emissions mode, CO2 also needs to be run in emissions mode.")
            self.cumulative_emissions['CO2|MAGICC AFOLU'] = np.cumsum(self.emissions['CO2|MAGICC AFOLU'])
            self.effective_radiative_forcing['Land use'] = (
                from_cumulative_co2(
                    self.cumulative_emissions['CO2|MAGICC AFOLU'],
                    self.emissions['CO2|MAGICC AFOLU'][0],
                    self.forcing_from_cumulative_co2_afolu,
                )
            )
        
        # 6. non-cumbersome way to insert prescribed forcing required here
        
        # 7. Do the main loop for feedback-dependent species and run energy balance model
        g1 = {}
        g0 = {}
        gas_boxes = {}
        ae_timestep = {}
        if self.run_mode['CH4'] is not None:
            ch4_lifetime = np.ones(n_timesteps) * np.nan
        for specie in ['CH4', 'CO2']:
            self.airborne_emissions[specie] = np.ones(n_timesteps) * np.nan
            ae_timestep[specie] = 0
            g0[specie], g1[specie] = calculate_g(lifetime[specie], partition_fraction[specie], iirf_horizon=iirf_horizon)
            gas_boxes[specie] = 0
            self.effective_radiative_forcing[specie] = np.ones(n_timesteps) * np.nan
            if self.run_mode[specie]=='emissions':
                self.concentration[specie] = np.ones(n_timesteps) * np.nan
                self.cumulative_emissions[specie] = np.cumsum(self.emissions[specie])
            elif self.run_mode[specie]=='concentrations':
                self.emissions[specie] = np.ones(n_timesteps) * np.nan
                self.cumulative_emissions[specie] = np.zeros_like(n_timesteps)
        
        erf_lasttimestep=0
        time_lasttimestep=self.time_minus1
        
        for i_time in range(n_timesteps):
            # 7a. ERF from feedback-dependent species
            
            # if self.run_mode['CH4']=='emissions':
            # Update methane concentrations
            alpha_lifetime = calculate_alpha(
                self.cumulative_emissions['CH4'][i_time],
                ae_timestep['CH4'],
                self.temperature[i_time],
                self.iirf_0["CH4"],
                self.iirf_cumulative["CH4"],
                self.iirf_temperature["CH4"],
                self.iirf_airborne["CH4"],
                g0['CH4'],
                g1['CH4'],
                iirf_max=self.iirf_max,
            )
            ch4_lifetime[i_time] = alpha_lifetime * lifetime['CH4']
            self.concentration['CH4'][i_time], gas_boxes['CH4'], ae_timestep['CH4'] = step_concentration(
                emissions['CH4'][i_time],
                gas_boxes['CH4'],
                ae_timestep['CH4'],
                self.burden_per_emission['CH4'],
                self.lifetime['CH4'],
                alpha_lifetime=alpha_lifetime,
                pre_industrial_concentration=self.pre_industrial_concentration['CH4'],
                timestep=time_deltas[i_time],
                partition_fraction=self.partition_fraction['CH4']
            )
            self.airborne_emissions['CH4'][i_time] = ae_timestep['CH4']
            
            # Update carbon dioxide concentrations
            alpha_lifetime = calculate_alpha(
                self.cumulative_emissions['CO2'][i_time],
                ae_timestep['CO2'],
                self.temperature[i_time],
                self.iirf_0["CO2"],
                self.iirf_cumulative["CO2"],
                self.iirf_temperature["CO2"],
                self.iirf_airborne["CO2"],
                g0['CO2'],
                g1['CO2'],
                iirf_max=self.iirf_max,
            )

            self.concentration['CO2'][i_time], gas_boxes['CO2'], ae_timestep['CO2'] = step_concentration(
                self.emissions['CO2'][i_time],
                gas_boxes['CO2'],
                ae_timestep['CO2'],
                self.burden_per_emission['CO2'],
                self.lifetime['CO2'],
                alpha_lifetime=alpha_lifetime,
                pre_industrial_concentration=self.pre_industrial_concentration['CO2'],
                timestep=time_deltas[i_time],
                partition_fraction=self.partition_fraction['CO2']
            )
            self.airborne_emissions['CO2'][i_time] = ae_timestep['CO2']

            # Calculate ERF from the "big 3" gases
            erf_ghg3 = meinshausen(
                {
                    "CO2": self.concentration['CO2'][i_time],
                    "CH4": self.concentration['CH4'][i_time],
                    "N2O": self.concentration['N2O'][i_time],
                },
                self.pre_industrial_concentration,
                self.tropospheric_adjustment,
            )

            for gas in ['CO2', 'CH4', 'N2O']:
                self.effective_radiative_forcing[gas][i_time] = erf_ghg3[gas]
            
            # Stratospheric water vapour goes here
            if self.run_mode['Stratospheric water vapour'] == 'emissions':
                self.effective_radiative_forcing['Stratospheric water vapour'] = linear_stwv(
                    self.effective_radiative_forcing
                )

            # Ozone goes here
            if self.run_mode['Ozone'] == 'emissions':
                erf_ozone = thornhill_skeie(
                    {
                        "CO": self.emissions['CO'][i_time],
                        "VOC": self.emissions['VOC'][i_time],
                        "NOx": self.emissions['NOx'][i_time],
                        
                    },
                    {gas:self.concentration[gas][i_time] for gas in ["CH4", "N2O"] + montreal_gas_list},
                    temperature=self.temperature[i_time]
                )
            
            for component in erf_ozone:
                self.effective_radiative_forcing[component][i_time] = erf_ozone[component]
            
            # Sum up all forcings
            erf_sum = 0
            # This list comprehension would be best to take out of this loop so we don't have to run it each time
            top_level_forcers = [forcer for forcer in self.effective_radiative_forcing if "|" not in forcer]
            for forcer in top_level_forcers:
                erf_sum = erf_sum + self.effective_radiative_forcing[forcer][i_time]
            self.effective_radiative_forcing['Total'][i_time] = erf_sum

            # 7b. temperatures
            ebm.add_forcing(
                np.array((erf_lasttimestep, erf_sum)),
                np.array((time_lasttimestep, self.time[i_time]))
            )
            ebm.run()
            self.temperature[i_time] = ebm.temperature[1, 0]

            ebm.temperature[0, :] = ebm.temperature[1, :]
            erf_lasttimestep = erf_sum
            time_lasttimestep = self.time[i_time]

In [ ]:
# Load up the CMIP6 tunings
df = pd.read_csv(
    os.path.join("..", "data", "calibration", "4xCO2_cummins.csv")
)
models = df['model'].unique()
models
params = {}
results = {}
for model in models:
    params[model] = {}
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        params[model][run] = {}
        params[model][run]['gamma_autocorrelation'] = df.loc[condition, 'gamma'].values[0]
        params[model][run]['ocean_heat_capacity'] = df.loc[condition, 'C1':'C3'].values.squeeze()
        params[model][run]['ocean_heat_transfer'] = df.loc[condition, 'kappa1':'kappa3'].values.squeeze()
        params[model][run]['deep_ocean_efficacy'] = df.loc[condition, 'epsilon'].values[0]
        params[model][run]['sigma_eta'] = df.loc[condition, 'sigma_eta'].values[0]
        params[model][run]['sigma_xi'] = df.loc[condition, 'sigma_xi'].values[0]
        params[model][run]['forcing_4co2'] = df.loc[condition, 'F_4xCO2'].values[0]

In [ ]:
solar_erf = forcing_from_input(
    '../src/fair21/defaults/data/ar6/solar_erf.csv',
    time=np.arange(1750.5, 2501),
    bounds_error=False,
    fill_value=0
)

volcanic_erf = forcing_from_input(
    '../src/fair21/defaults/data/ar6/volcanic_erf.csv',
    time=np.arange(1750.5, 2501),
    bounds_error=False,
    fill_value=0
)

In [ ]:
for model in tqdm(models):
    for run in df.loc[df['model']==model, 'run']:
        condition = '{}_{}'.format(model, run)
        #ebm = EnergyBalanceModel(**params[model][run])
        scen = FAIR(**params[model][run])
        for specie in gas_list + slcf_list + ['CO2|MAGICC AFOLU']:
            scen.add_emissions({specie: emissions[specie]})
        scen.add_forcing(
            {
                'Volcanic': volcanic_erf,
                'Solar': solar_erf
            }
        )
        scen.define_time(np.arange(1750.5, 2501))
        #scen.prescribe_temperature(ssp245_temperature_rfmip)
        scen.run()
        results[condition] = scen.temperature

In [ ]:
for condition in results:
    pl.plot(np.arange(1750.5, 2501), results[condition])

In [ ]:
for condition in results:
    pl.plot(np.arange(1850.5, 2101), (results[condition]-results[condition][100:151].mean())[100:351])
pl.grid()

In [ ]:
pl.plot(scen.effective_radiative_forcing['Aerosol'])

In [ ]:
scen.effective_radiative_forcing.keys()

In [ ]:
#pl.plot(scen.effective_radiative_forcing['Total'])
sumcat = [forcer for forcer in scen.effective_radiative_forcing if forcer not in ['Total', 'Ozone|Emitted Gases|CH4', 'Ozone|Emitted Gases|N2O', 'Ozone|Emitted Gases|Montreal Gases', 'Ozone|Emitted Gases|VOC', 'Ozone|Emitted Gases|NOx', 'Ozone|Emitted Gases|CO', 'Ozone|Emitted Gases', 'Ozone|Temperature Feedback', 'Aerosol|Aerosol-radiation interactions|Sulfur', 'Aerosol|Aerosol-radiation interactions|BC', 'Aerosol|Aerosol-radiation interactions|OC', 'Aerosol|Aerosol-radiation interactions|NH3', 'Aerosol|Aerosol-radiation interactions', 'Aerosol|Aerosol-cloud interactions']]
print(sumcat)
erf_bottom_up = 0
for forcer in sumcat:
    erf_bottom_up = erf_bottom_up + scen.effective_radiative_forcing[forcer]
pl.plot(erf_bottom_up - scen.effective_radiative_forcing['Total'])

In [ ]:
# grab some concentrations to compare
concentration_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
for gas in gas_list:
    gas_rcmip_name = gas.replace("-", "")
    concentration_rcmip[gas] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()

In [ ]:
fig, ax = pl.subplots(6, 8, figsize=(16,16))
for igas, gas in enumerate(scen.concentration.keys()):
    iy = igas % 8
    ix = igas // 8
    ax[ix, iy].plot(np.arange(1750.5, 2501), scen.concentration[gas], label='FaIR 2.1')
    ax[ix, iy].plot(np.arange(1750.5, 2501), concentration_rcmip[gas], label='History + MAGICC6')
    ax[ix, iy].set_title(gas)
fig.tight_layout()

In [ ]:
pl.plot(scen.time, scen.temperature)

In [ ]:
pl.plot(scen.time, scen.effective_radiative_forcing['Total'])

In [ ]:
dir(scen)

In [ ]:
scen.temperature[270]